In [ ]:
from sage.all import *
import numpy as np
from matplotlib import pyplot as plt

In [1]:
class NBRW:
    """A class to store several relevant attributes of a graph G in Non-Backtracking Random Walks"""
    
    def __init__(self, G, pinwheel=False):
        """Accepts a a sage graph"""
        
        self.G = G  # store graph
        self.m = len(G.edges())  # number of edges
        self.n = len(G.vertices())  # number of vertices
        
        self.S = self.S_mat()
        self.T = self.T_mat()
        self.tau = self.tau_mat()
        self.B = self.B_mat()
        self.Dinv = self.D_inv(pinwheel)
        
        self.Pnb = self.nb_trans_mat()
        self.Znb = self.Znb_mat()
        self.Wnb = self.Wnb_mat()
        self.Mev = self.M_ev_NBRW()
        self.Znb_e = self.Znb_e_mat()
        self.M = self.M_mat()
        self.Mv_nb = self.nb_vertex_mfpt()
        self.Wv = self.Wv_mat()
        self.Rv = self.mrt_NBRW_vertex()
        self.Knb_v = self.nb_vertex_kemeny_mfpt(pinwheel)
        self.P = self.trans_mat_sage()
        self.K_vertex = self.kemeny_vertex()
        self.K_edge = self.kemeney_edge()
        self.Mnb_e = self.nb_mfpt_mat()
        self.Knb_e = self.kemeney_nb_edge()

        self.De = self.De_mat()
        self.De_inv = np.linalg.inv(self.De)
        self.Pe = self.De_inv @ self.S @ self.T
        self.Re = self.Re_mat()
        
        self.Knb_v2 = np.trace(self.Znb) - 1
        self.Knb_v3 = self.nb_vertex_kemeny_trace()

        self.volG = 2 * len(self.G.edges())
        if pinwheel:
            self.pi = np.array([d / self.volG for d in sorted(self.G.degree(), reverse=True)])
        else:
            self.pi = np.array([d / self.volG for d in self.G.degree()])

        self.Z = self.fund_mat()
        self.Mv = self.mfpt_mat(size=self.n, Z=self.Z, pi=self.pi)

        self.Italian_Mv_nb = self.italian_hitting_times_mat()

        # New Edge Space Stuff
        self.pi_e = np.ones(2*self.m) / (2*self.m)
        self.We = np.outer(np.ones(2*self.m), self.pi_e)
        self.Z_e = np.linalg.inv(np.eye(2*self.m) - self.Pe + self.We)
        self.M_e = self.mfpt_mat(size=2*self.m, Z=self.Z_e, pi=self.pi_e)

        
    
    def show(self):
        """Prints the graph"""
        self.G.show()
    
    def S_mat(self):
        """Returns matrix S such that ST = C (TS=A)
        S maps from edge space to vertex space
        S is a 2m x n matrix"""
        
        edges = list(self.G.edges())
        
        S = matrix.zero(2*self.m, self.n)
        for x in range(self.n):
            # iterate through nodes
            for j in range(self.m):
                # iterate through edges
                if edges[j][1] == x:
                    S[j, x] = 1
                if edges[j][0] == x:
                    S[j+self.m, x] = 1
        return S
    
    def T_mat(self):
        """Returns T matrix such that ST = C (TS=A)
        Maps from vertex to edge
        n x 2m"""
        
        edges = list(self.G.edges())
        
        T = matrix.zero(self.n, 2*self.m)
        for x in range(self.n):
            # iterate through nodes
            for j in range(self.m):
                # iterate through edges
                if edges[j][0] == x:
                    T[x,j] = 1
                if edges[j][1] == x:
                    T[x, j+self.m] = 1
        
        return T
    
    def tau_mat(self):
        """Return matrix tau such that ST - tau = B
        2m x 2m
        Switches directed edges
        """
        
        edges = list(self.G.edges())
        
        zero = matrix.zero(self.m)  # m x m zeros
        I = identity_matrix(self.m)  # m x m identity
        
        t = block_matrix([[zero, I], [I, zero]])
        
        return t  # like an identity with opposite diagonal
    
    def B_mat(self):
        """Return matrix B = ST - tau
        non-backtracking edge adjacency matrix"""
        
        return self.S * self.T - self.tau
    
    def nb_trans_mat(self):
        """return NBRW edge space transition probability matrix
        P_{nb} = (D-I)^{-1}B"""
        b = self.B
        row_sums = sum(b)
        row_sums = [i ^ -1 for i in row_sums]
        return b * diagonal_matrix(row_sums)
    
    def D_inv(self, pinwheel=False):
        """D^{-1}"""
        if pinwheel:
            return np.diag([1 / d for d in sorted(self.G.degree(), reverse=True)])
        else:    
            return np.diag([1 / d for d in self.G.degree()])
    
    def trans(self):
        """vertex space transition matrix of a simple random walk"""
        D = self.D
        return D @ self.G.adjacency_matrix()
    
    def nb_hitting_times(self, j):
        """
        return list (np.array()) m_{i, j} where m is
        NB hitting time from i to j for all i, with j fixed.

        THIS IS ACTUALLY I THINK nb hitting time m_{e, j}. 2m vector of hitting times from
        the edge e to the vertex j. (SEE Theorem 4.3 I believe.)

        i.e. the columns of M_{ev}
        """
        # Ask Adam what's going on here!
        m = len(self.G.edges())
        n = len(self.G)
        P = self.nb_trans_mat()
        Gedges = [(g[0], g[1]) for g in self.G.edges()]
        Gedges.extend([(g[1], g[0]) for g in self.G.edges()])
        entries_to_delete = []

        # Find the rows/columns to delete
        for i in range(len(Gedges)):
            if Gedges[i][0] == j:
                entries_to_delete.append(i)
        P_new = np.delete(P, entries_to_delete, axis=0)
        P_new = np.delete(P_new, entries_to_delete, axis=1)

        # Keep a list of what was NOT deleted, to construct the full vector later
        remaining_entries = [i for i in range(2 * m)]
        for k in entries_to_delete:
            remaining_entries.remove(k)

        # Solve (I - P)x = 1
        ones = np.array([1] * len(P_new))
        tau_vec = np.linalg.solve(np.identity(len(P_new)) - P_new, ones)

        # recreate full tau_vec, putting the 0's back in where necessary
        full_tau = np.array([0.0] * (2 * m))
        for i in range(len(remaining_entries)):
            full_tau[remaining_entries[i]] = tau_vec[i]

        return full_tau
    
    def M_ev_NBRW(self):
        """
        return the matrix $M_{ev}$, that is, the hitting times (mean first passage times)
        of the nonbacktracking random walk
        """
        m = len(self.G.edges())
        n = len(self.G)
        Mev = np.zeros((2 * m, n))
        for i in range(n):
            Mvec = self.nb_hitting_times(i)
            for j in range(2 * m):
                Mev[j][i] = Mvec[j]

        return Mev
    
    def P_hat(self):
        """
        (D-I)^{-1}A.
        I should probably not take an inverse since its so easy but whatever. change as needed
        """
        return np.diag([1 / (d - 1) for d in G.degree()]) @ G.adjacency_matrix()


    def P_mat(self):
        """
        D^{-1}A
        """
        return np.diag([1 / d for d in G.degree()]) @ G.adjacency_matrix()

    
    def Znb_mat(self):
        """
        A guess at what Znb should be. Seems to work pretty close to what we want numerically. Matches exactly with
        edge transitive graphs. Usually close with all other graphs we've tried.
        """
        n = len(self.G)
        m = len(self.G.edges())
        Dinv = np.diag([1 / d for d in self.G.degree()])
        T = self.T
        S = self.S
        Wnb = np.ones((2 * m, 2 * m)) / (2 * m)
        Wv = np.outer(np.ones(n), [d / (2 * m) for d in self.G.degree()])

        return np.eye(n) + Dinv @ T @ (np.linalg.inv(np.eye(2 * m) - self.nb_trans_mat() + Wnb)) @ S - Wv

    
    def Wnb_mat(self):
        return np.ones((2 * self.m, 2 * self.m)) / (2 * self.m)
    
    
    def Znb_e_mat(self):
        I = identity_matrix(2*self.m)
        return np.linalg.inv(I - self.Pnb + self.Wnb)
    
    
    def M_mat(self):
        """
        Matrix M as in (4.4) in the Hitting Times Paper
        (Basically the start point operator divided by the row sums)
        """
        M = self.T.numpy()
        # Divide by row sums
        return M / M.sum(axis=1, keepdims=1)
    
    
    def nb_vertex_mfpt(self):
        """
        Get the matrix (np.array probably) where M_{i,j} is the mfpt i -> j.
        That is, $M_v^{nb}$

        For now, assumes vertices are {0, 1, ..., n-1}
        """
        n = self.n
        MFPT = np.zeros((n, n))

        for i in range(n):
            MFPT_col_i = self.M @ self.nb_hitting_times(i)
            for j in range(n):
                MFPT[j][i] = MFPT_col_i[j]

        return MFPT
    
    def Wv_mat(self):
        return np.outer(np.ones(self.n), [d / (2 * self.m) for d in self.G.degree()])
    
    def mrt_NBRW_vertex(self):
        """
        Diagonal matrix of mean return times of a NBRW.
        See Equations below (4.8) in hitting times paper.

        Note: This should be the same Mean Return Times as SRW I believe which might just be 1/\pi_j
              if I remember correctly as I'm typing this note
        """
        Dinv = np.diag([1 / d for d in self.G.degree()])
        T = self.T
        Pnb = self.Pnb
        Mev = self.Mev
        return np.eye(len(self.G)) + np.diag(np.diag(Dinv @ T @ Pnb @ Mev))
    
    def trans_mat_sage(self):
        """vertex space transition matrix of a simple random walk"""
        return self.Dinv @ self.G.adjacency_matrix()
    
    
    def Pv_NBRW(self, k):
        """
        Return the vertex space transition probability matrix for the nonbacktracking random walk for the kth step
        """
        if k == 0:
            return np.eye(len(self.G))

        if k == 1:
            return trans_mat_sage()

        # If k > 1
        return self.Dinv @ self.T @ (np.linalg.matrix_power(self.Pnb, k - 1)) @ self.S

    def nb_vertex_kemeny_mfpt(self, pinwheel=False):
        """
        Use these hitting times to get Kemeny's constant
        """
        volG = 2 * len(self.G.edges())
        if pinwheel:
            steady = np.array([d / volG for d in sorted(self.G.degree(), reverse=True)])
        else:
            steady = np.array([d / volG for d in self.G.degree()])
        return steady @ self.Mv_nb @ steady
    
    def kemeny_vertex(self):
        """SRW Vertex Kemeny"""
        eigvals = np.array(sorted(np.linalg.eigvals(self.P))[:-1])  # eigenvalues of P transition matrix without \lambda_max = 1
        return np.sum(1/(1-eigvals))
        
    def kemeney_edge(self):
        """SRW Edge Space Kemeny Theorem 2.9"""
        return self.K_vertex + 2*self.m - self.n
    
    def kemeney_nb_edge(self):
        """nb kemeny's for a graph in the edge space"""
        return np.trace(self.Znb_e) - 1
    
    def De_mat(self):
        De = np.zeros((2*self.m, 2*self.m))
        deg = self.G.degree()
        for idx, tup in enumerate(self.G.edges()):
            i, j, _ = tup
            De[idx, idx] = deg[j]
            De[self.m + idx, self.m + idx] = deg[i]
        return De
    
    def nb_vertex_kemeny_trace(self):
        return self.Knb_e - np.trace(self.De_inv @ self.Znb_e) + self.n * (1 - 1/(2*self.m)) - 2*self.m + self.n + np.trace(self.De_inv @ self.tau @ self.Znb_e)

    def nb_fund_mat(self, i):
        """compute (I-Q)^-1 where Q = P[i,i], the transition probability matrix
        with deleted cols/rows i"""
        G = self.G
        P = self.nb_trans_mat()
        P = np.delete(P, i, 0)
        P = np.delete(P, i, 1)
        return np.linalg.inv(np.identity(len(P)) - P)


    def nb_mfpt_mat(self):
        """Mean First Passing Time (Hitting Time) matrix in the edge space for
        a Non-Backtracking Random Walk."""
        G = self.G
        P = np.array(self.nb_trans_mat())
        volG = len(P)

        M = np.zeros((volG, volG))
        for i in range(volG):
            #get N for all directed edges
            Ni = self.nb_fund_mat(i)
            for j in range(len(Ni)):
            #get all row sums, careful to put in correct place
                if j < i:
                    #if j index before i, no problem
                    M[j,i] = np.sum(Ni[j,:])
                if j >= i:
                    #takes care of reindexing
                    M[j+1,i] = np.sum(Ni[j,:])

        return M

    def fund_mat(self):
        return np.linalg.inv(np.eye(self.n) - self.P + self.Wv)

    def mfpt_mat(self, size, Z, pi):
        M = np.zeros((size, size))
        for i in range(size):
            for j in range(size):
                if i != j:
                    M[i, j] = (Z[j, j] - Z[i, j]) / pi[j]
        
        return M
    

    def italian_hitting_times_mat(self):
        diag_alpha = 1/(2*self.m) * np.eye(2*self.m) + 1/(2*self.m) * np.diag(np.diag(self.Pnb @ self.Mev @ self.T))
        beta = np.diag(self.Dinv @ self.T @ self.Mnb_e @ diag_alpha @ self.T.T)
        Mv_nb = self.Dinv @ self.T @ self.Mnb_e @ diag_alpha @ self.T.T - np.outer(np.ones(len(beta)), beta)
        return Mv_nb
    
    def Re_mat(self):
        return 2*self.m * np.eye(2*self.m)


In [ ]:
"""Code to generate various graph families. From Adam Knudson"""

def cycle_barbell(k, a, b):
    """
    path on k verties, the end two are shared with a cycle of size a and a cycle of size b
    so |V(G)| = a + b + c - 2
    """
    CB = Graph(k + a + b - 2)
    
    # make the a-cycle
    for i in range(a - 1):
        CB.add_edge(i, i+1)
    CB.add_edge(0, a-1)

    for i in range(k-1):
        CB.add_edge(a-1+i, a+i)
    
    for i in range(b-1):
        CB.add_edge(a+k-2+i, a+k-1+i)
    CB.add_edge(a+k-2, a+b+k-3)


    return CB


def necklace(k):
    """
    |V| = n = 4k + 2
    k is number of "beads" including the end ones which are a little different
    (so only works if k\geq2)
    """
    n = 4*k + 2
    G = Graph(n)
    #first bead
    G.add_edges([(0,1), (0, 2), (0,3), (1,2), (1, 3), (2,4), (3,4)])
    #last bead
    G.add_edges([(n-1, n-2), (n-1, n-3), (n-1, n-4), (n-2, n-3), (n-2, n-4), (n-3, n-5), (n-4, n-5)])
    #All the middle ones
    for i in range(1, k-1):
        G.add_edge(4*i, 4*i + 1)
        G.add_edges([(4*i+1, 4*i + 2), (4*i+1, 4*i+3), (4*i+2, 4*i+3), (4*i+2, 4*(i+1)), (4*i+3, 4*(i+1))])
        
    #Final edge
    G.add_edge(n - 6, n-5)
    return G

def pinwheel(cycle_sizes: list[int]):
    """Accepts a list of integers called cycle_sizes, which will determine the number 
    of cycles and also their length."""
    # Create an empty graph
    G = Graph()
    
    # Add the central vertex
    central_vertex = 0
    
    # Track the next available vertex id
    next_vertex_id = 1
    
    for size in cycle_sizes:
        # Create a cycle graph with an additional vertex for the central vertex
        C = Graph()
        C.add_vertex(central_vertex)
        
        # Add the vertices for the cycle
        cycle_vertices = [next_vertex_id + i for i in range(size - 1)]
        C.add_vertices(cycle_vertices)
        
        # Add edges to form the cycle including the central vertex
        for i in range(size - 1):
            if i == 0:
                # Connect the central vertex to the first vertex in the cycle
                C.add_edge(central_vertex, cycle_vertices[i])
            else:
                # Connect the previous vertex to the current vertex
                C.add_edge(cycle_vertices[i-1], cycle_vertices[i])
        
        # Complete the cycle
        C.add_edge(cycle_vertices[-1], central_vertex)
        
        # Add the cycle graph to G
        G.add_vertices(C.vertices())
        G.add_edges(C.edges())
        
        # Update the next available vertex id
        next_vertex_id += size - 1
    
    return G



In [ ]:
def N(A):
    two_m, _ = A.shape
    return two_m * np.max(np.abs(A))

def Bound(G):
    """accepts NBRW  object"""
    m = G.m
    I = np.eye(2*m)
    top = N(I) - (N(I) - 1) * N(G.Pnb - G.Wnb)
    bot = 1 - N(G.Pnb - G.Wnb)
    return top / bot

In [ ]:
import random as r
def order_of_magnitude_on_cycle_barbell_graphs():
    """We want to establish/ conjecture that Knb_v is O(n^2), whereas Kv is O(n^3). The latter is known"""

    plt.figure(figsize=(10, 10))
    
    v, nb_v, nb_v2, n = [], [], [], []
    options = list(range(3, 41))
    for _ in range(50):
        i, j, k = r.sample(options, 3)
        n.append(i + j + k)
        g = cycle_barbell(i, j, k)
        g = NBRW(g)
        v.append(g.K_vertex)
        nb_v.append(g.Knb_v2)
        nb_v2.append(g.Knb_v)
    
    # plt.plot(dom, v, label=r"$K^v$")
    plt.scatter(n, nb_v, c='skyblue', label=r"$K_{nb}^v$")
    plt.scatter(n, nb_v2, c='k', label=r"$K_{nb}^v$, $\pi^TM\pi$")
    plt.legend()
    plt.title("Random Cycle Barbells")
    plt.xlabel("n")
    plt.ylabel("Kemeny's Constant")
    plt.show()
    
    

In [ ]:
import random as r
def order_of_magnitude_on_barbell_graphs():
    """We want to establish/ conjecture that Knb_v is O(n^2), whereas Kv is O(n^3). The latter is known"""

    plt.figure(figsize=(10, 10))
    
    v, nb_v, nb_v2, n = [], [], [], []
    options = list(range(3, 21))
    for _ in range(5):
        i, j = r.sample(options, 2)
        n.append(i + j)
        g = graphs.BarbellGraph(i, j)
        g = NBRW(g)
        v.append(g.K_vertex)
        nb_v.append(g.Knb_v2)
        nb_v2.append(g.Knb_v)
    
    # plt.plot(dom, v, label=r"$K^v$")
    plt.scatter(n, nb_v, c='skyblue', label=r"$K_{nb}^v$")
    plt.scatter(n, nb_v2, c='k', label=r"$K_{nb}^v$, $\pi^TM\pi$")
    plt.legend()
    plt.title("Random Barbells")
    plt.xlabel("n")
    plt.ylabel("Kemeny's Constant")
    plt.show()

In [ ]:
def order_of_magnitude_on_barbell_graphs_deterministic():

    plt.figure(figsize=(10, 10))

    nb_v, nb_v2 = [], []
    ns = [5, 8, 10, 12, 15, 18, 20]
    for n in ns:
        g = graphs.BarbellGraph(n, 1)
        g = NBRW(g)
        nb_v.append(g.Knb_v2)
        nb_v2.append(g.Knb_v)

    plt.scatter(ns, nb_v, c='skyblue', label=r"$K_{nb}^v$")
    plt.scatter(ns, nb_v2, c='k', label=r"$K_{nb}^v$, $\pi^TM\pi$")
    plt.legend()
    plt.title("Random Barbells")
    plt.xlabel("n")
    plt.ylabel("Kemeny's Constant")
    plt.show()


In [ ]:
def trace_upper_bound(G):
    tau = G.tau
    m = G.m
    Z = G.Znb_e
    Z_ = 1/2 * (Z + Z.T)
    t_max = max(np.linalg.eigvals(tau))
    z__min = min(np.linalg.eigvals(Z_))
    return t_max * np.trace(Z) - z__min * (2*m* t_max - np.trace(tau))

In [ ]:
from scipy import linalg as la

def Drazin(A, tol=1e-4):
    """Accepts a numpy nd-array and computes the Drazin inverse
    
    Parameters:
        A ((n,n) ndarray): An nxn matrix.
        tol (float): A float close to zero to classify 0 eigenvalues

    Returns:
        AD ((n,n) ndarray): The nxn Drazin Inverse of A
    """
    n, n = A.shape
    # This is some algorithm to compute the Drazin Inverse I found in an archived ACME Lab
    T1, Q1, k1 = la.schur(A, sort=lambda x: abs(x) > tol)
    T2, Q2, k2 = la.schur(A, sort=lambda x: abs(x) <= tol)
    U = np.hstack((Q1[:, :k1], Q2[:, :n-k1]))
    U_inv = la.inv(U)
    V = U_inv @ A @ U
    Z = np.zeros((n, n))
    if k1 != 0:
        Z[:k1, :k1] = la.inv(V[:k1, :k1])
    return U @ Z @ U_inv


def verify_if_drazin(A, k, AD):
    """Verify that a matrix AD is the Drazin inverse of A
    
    Parameters:
        A ((n,n) ndarray): An nxn matrix.
        AD ((n,n) ndarray): An nxn candidate for Drazin inverse of A
        k (int): the index of A

    Returns:
        (bool) True of AD is the Drazin inverse of A, False otherwise
    """
    is_drazin = True
    # Test all cases that uniquely determine the drazin inverse of A
    if not np.allclose(A@AD, AD@A, rtol=1e-3):  
        return False, 1
    B = np.linalg.matrix_power(A, k)  # only compute this once, A^k
    if not np.allclose(B@A@AD, B, rtol=1e-3):
        return False, 2
    if not np.allclose(AD @ A @ AD, AD, rtol=1e-3):
        return False, 3
    
    return is_drazin  # otherwise everything passed, so return true

def index(A, tol=1e-5):
    """Compute the index of the matrix A 
    
    Parameters:
        A ((n,n) ndarray): An nxn matrix

    Returns:
        k (int): The index of A
    """

    # test for non-singularity
    if not np.allclose(la.det(A), 0):
        return 0

    n = len(A)
    k = 1
    Ak = A.copy()
    while k <= n:
        r1 = np.linalg.matrix_rank(Ak)
        r2 = np.linalg.matrix_rank(np.dot(A, Ak))
        if r1 == r2:  # equivalent to checking dimension of nullspaces
            return k
        Ak = np.dot(A, Ak)
        k += 1
    
    return k


In [ ]:
from matplotlib import pyplot as plt
def plot_knbv_kv(n: int):
    """Code that plots both Knb^v(G) and K^v(G) for comparison. 
    Considers all complete graphs of minimum degree 2.
    n = 8 take several minutes to run.

    parameters:
        k (int): max number of vertices to run on. 
    """
    for k in range(3, n+1):
        for g in graphs.nauty_geng(f"{k} -c -d2"):
            pass

    ## Come back to. I'm not really sure the best way to plot this. Should vertices be on the x-axis? 

In [ ]:
def _get_Pnb_k(G, k):
    return G.Dinv @ np.array(G.T) @ np.linalg.matrix_power(G.Pnb, k-1) @ np.array(G.S)


def estimate_mu(G, max_iters=1000, tol=1e-5):
    """estimate the eigenvalues of Pnb^v
    compute the lim_{k\to\infty} (mu_i^{(k)})^{1/k} for each i.
    parameters:
        G (NBRW) : the graph G as the NBRW class object
    """
    Pnb_1 = _get_Pnb_k(G, 1)
    mu = np.flip(np.sort(np.linalg.eigvals(Pnb_1)))
    for k in range(2, max_iters+1):
        Pnb_k = _get_Pnb_k(G, k)
        mu_k = np.abs(np.flip(np.sort(np.linalg.eigvals(Pnb_k))))
        # print(mu_k)
        mu_k[np.abs(mu_k) < 1e-5] = 0.
        # print(mu_k)
        mu_k = mu_k ** (1/k)
        # print(mu_k)

        print()
        print(mu_k)
        print(mu)
        print(mu_k - mu)
        print(np.all(mu_k - mu < tol))
        if np.all(mu_k - mu < tol):
            print("Converged?", k)
            return mu_k
        else:
            mu = mu_k
    
    print(mu, mu_k, sep="\n")
    print(mu_k - mu)
    print(mu_k - mu < tol)
    print("Probably didn't converge")
    return mu, mu_k


In [ ]:
def is_diag_dom(A):
    """return True if A is diagonally dominant, False otherwise
    Parameters:
        A (ndarray) : matrix in question
    Returns:
        is_dd (bool) : True if dd False else"""
    diags = np.diag(A)
    row_sums = np.sum(np.abs(A), axis=1)
    deleted_rs = row_sums - diags
    if np.all(deleted_rs <= diags):
        return True
    return False

In [ ]:
import re

class cb_expr_eval():
    def __init__(self, expr: str) -> None:
        # Preprocess the Mathematica expression to convert it into a Python-compatible format
        expr = expr.replace("^", "**")  # Replace Mathematica exponentiation with Python's exponentiation
        expr = self.add_multiplication(expr)  # Add explicit multiplication
        self.expr = expr

    def add_multiplication(self, expr: str) -> str:
        # Add explicit multiplication where needed
        # Insert * between number and variable or variable and variable (e.g., 2a -> 2*a, ab -> a*b)
        expr = re.sub(r'(\d)([a-zA-Z])', r'\1*\2', expr)
        expr = re.sub(r'([a-zA-Z])(\d)', r'\1*\2', expr)
        expr = re.sub(r'([a-zA-Z])([a-zA-Z])', r'\1*\2', expr)
        return expr

    def eval(self, a: int, k: int, b: int) -> float:
        # Convert the input values to strings
        a_str, k_str, b_str = str(a), str(k), str(b)
        # Replace variables in the expression with their respective values
        new_expr = self.expr.replace("a", a_str).replace("k", k_str).replace("b", b_str)
        # Evaluate the expression
        return eval(new_expr)


In [ ]:
def conjected_largest_graph_family(n: int, definition: str = "one") -> Graph:
    """On n vertices, finds the graph with the largest kemeny's constant
        one -> pi.T @ M @ pi
        two -> trace(Znb) - 1
        three -> Knb_e - (2m - n)"""
    max_kemeny = 0
    for g in graphs.nauty_geng(f"{n} -c -d2"):
        try:
            g = NBRW(g)
            if definition == "one":
                val = g.Knb_v
            elif definition == "two":
                val = g.Knb_v2
            else: 
                val = g.Knb_e - (2*g.m - g.n)
            
            if val > max_kemeny:
                max_kemeny = val
                max_graph = g
        except np.linalg.LinAlgError:
            pass
    return max_graph